In [1]:
import pandas as pd
import csv
import re
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}/calendar.csv'

sf_file = file_url.format('sf_airbnb')
ok_file = file_url.format('ok_airbnb')
sc_file = file_url.format('sc_airbnb')
scz_file = file_url.format('scz_airbnb')

In [3]:
pricing_sf = pd.read_csv(sf_file, parse_dates=['date'])
pricing_ok = pd.read_csv(ok_file, parse_dates=['date'])
pricing_sc = pd.read_csv(sc_file, parse_dates=['date'])
pricing_scz = pd.read_csv(scz_file, parse_dates=['date'])

In [4]:
print(pricing_sf.shape, pricing_ok.shape, pricing_sc.shape, pricing_scz.shape)

(2420680, 4) (1057770, 4) (2068820, 4) (573050, 4)


In [5]:
frames = [pricing_sf, pricing_ok, pricing_sc, pricing_scz]
keys = ['pricing_sf', 'pricing_ok', 'pricing_sc', 'pricing_scz']

In [6]:
df_pricing = pd.concat(frames, keys=keys)

In [7]:
df_pricing.listing_id.nunique()

16768

In [8]:
df_pricing.head(20)

listing_id       date available price
pricing_sf 0          958 2019-08-05         f   NaN
           1          958 2019-08-04         f   NaN
           2          958 2019-08-03         f   NaN
           3          958 2019-08-02         f   NaN
           4          958 2019-08-01         f   NaN
           5          958 2019-07-31         f   NaN
           6          958 2019-07-30         f   NaN
           7          958 2019-07-29         f   NaN
           8          958 2019-07-28         f   NaN
           9          958 2019-07-27         f   NaN
           10         958 2019-07-26         f   NaN
           11         958 2019-07-25         f   NaN
           12         958 2019-07-24         f   NaN
           13         958 2019-07-23         f   NaN
           14         958 2019-07-22         f   NaN
           15         958 2019-07-21         f   NaN
           16         958 2019-07-20         f   NaN
           17         958 2019-07-19         f   NaN
           18         958 2019-07-18         f   NaN
           19         958 2019-07-17         f   NaN

In [9]:
# Filter down to houses that are available and have listing prices

df_pricing_smaller = df_pricing[df_pricing['available']=='t']
df_pricing_smaller = df_pricing_smaller.copy()

In [10]:
df_pricing_smaller.shape

(2853357, 4)

In [11]:
df_pricing_smaller.listing_id.nunique()

14770

In [12]:
df_pricing_smaller.date.describe(include='all')

count                 2853357
unique                    389
top       2018-11-03 00:00:00
freq                    10393
first     2018-08-06 00:00:00
last      2019-08-29 00:00:00
Name: date, dtype: object

In [13]:
df_pricing_smaller.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [14]:
def currency_to_float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [15]:
# Convert price to float format

df_pricing_smaller['price'] = df_pricing_smaller['price'].apply(lambda x: currency_to_float(x))

In [16]:
# Add a month column based on date

df_pricing_smaller['month'] = df_pricing_smaller['date'].dt.month

In [17]:
# Add a day_of_week column based on Date. Use default format: Monday=0, Sunday=6

df_pricing_smaller['day_of_week'] = df_pricing_smaller['date'].dt.weekday

In [18]:
df_pricing_smaller.head()

listing_id       date available  price  month  day_of_week
pricing_sf 63         958 2019-05-02         t  181.0      5            3
           64         958 2019-05-01         t  181.0      5            2
           65         958 2019-04-30         t  181.0      4            1
           66         958 2019-04-29         t  181.0      4            0
           67         958 2019-04-28         t  181.0      4            6

In [19]:
df_pricing_smaller.shape

(2853357, 6)

In [20]:
df_pricing_smaller.describe(include ='all')

,listing_id,date,available,price,month,day_of_week
count,2.853357e+06,2853357,2853357,2.853357e+06,2.853357e+06,2.853357e+06
unique,NaN,389,1,NaN,NaN,NaN
top,NaN,2018-11-03 00:00:00,t,NaN,NaN,NaN
freq,NaN,10393,2853357,NaN,NaN,NaN
first,NaN,2018-08-06 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2019-08-29 00:00:00,NaN,NaN,NaN,NaN
mean,1.405523e+07,NaN,NaN,2.132333e+02,6.571592e+00,2.988646e+00
std,8.563144e+06,NaN,NaN,7.035042e+02,3.629179e+00,2.003061e+00
min,9.580000e+02,NaN,NaN,1.000000e+01,1.000000e+00,0.000000e+00
25%,6.453774e+06,NaN,NaN,8.000000e+01,3.000000e+00,1.000000e+00


In [21]:
# Count how many months each house was listed in the last year

df_month_count = df_pricing_smaller.groupby(['listing_id']).month.nunique().reset_index()
df_month_count.rename(columns={'month':'month_count'}, inplace=True)
df_month_count.head()

,listing_id,month_count
0,958,10
1,3083,5
2,3264,3
3,5739,2
4,5858,12


In [22]:
# Count how many days of week each house was listed in the last year

df_day_of_week_count = df_pricing_smaller.groupby(['listing_id']).day_of_week.nunique().reset_index()
df_day_of_week_count.rename(columns={'day_of_week':'day_of_week_count'}, inplace=True)
df_day_of_week_count.head()

,listing_id,day_of_week_count
0,958,7
1,3083,7
2,3264,7
3,5739,7
4,5858,7


In [23]:
df_pricing_with_counts = pd.merge(left=df_pricing_smaller, 
                                  right=df_month_count,
                                  how='left', 
                                  left_on='listing_id', 
                                  right_on='listing_id')
df_pricing_with_counts.head()

,listing_id,date,available,price,month,day_of_week,month_count
0,958,2019-05-02,t,181.0,5,3,10
1,958,2019-05-01,t,181.0,5,2,10
2,958,2019-04-30,t,181.0,4,1,10
3,958,2019-04-29,t,181.0,4,0,10
4,958,2019-04-28,t,181.0,4,6,10


In [24]:
df_pricing_with_counts = pd.merge(left=df_pricing_with_counts, 
                                  right=df_day_of_week_count,
                                  how='left', 
                                  left_on='listing_id', 
                                  right_on='listing_id')
df_pricing_with_counts.head()

,listing_id,date,available,price,month,day_of_week,month_count,day_of_week_count
0,958,2019-05-02,t,181.0,5,3,10,7
1,958,2019-05-01,t,181.0,5,2,10,7
2,958,2019-04-30,t,181.0,4,1,10,7
3,958,2019-04-29,t,181.0,4,0,10,7
4,958,2019-04-28,t,181.0,4,6,10,7


In [25]:
df_pricing_with_counts.describe(include='all')

,listing_id,date,available,price,month,day_of_week,month_count,day_of_week_count
count,2.853357e+06,2853357,2853357,2.853357e+06,2.853357e+06,2.853357e+06,2.853357e+06,2.853357e+06
unique,NaN,389,1,NaN,NaN,NaN,NaN,NaN
top,NaN,2018-11-03 00:00:00,t,NaN,NaN,NaN,NaN,NaN
freq,NaN,10393,2853357,NaN,NaN,NaN,NaN,NaN
first,NaN,2018-08-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2019-08-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.405523e+07,NaN,NaN,2.132333e+02,6.571592e+00,2.988646e+00,1.020866e+01,6.995392e+00
std,8.563144e+06,NaN,NaN,7.035042e+02,3.629179e+00,2.003061e+00,2.696574e+00,1.206497e-01
min,9.580000e+02,NaN,NaN,1.000000e+01,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,6.453774e+06,NaN,NaN,8.000000e+01,3.000000e+00,1.000000e+00,9.000000e+00,7.000000e+00


In [26]:
df_pricing_with_counts.price.describe(percentiles=[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95])

count    2.853357e+06
mean     2.132333e+02
std      7.035042e+02
min      1.000000e+01
5%       4.100000e+01
10%      5.400000e+01
25%      8.000000e+01
50%      1.350000e+02
75%      2.250000e+02
90%      3.850000e+02
95%      5.500000e+02
max      1.024520e+05
Name: price, dtype: float64

In [27]:
# Remove outliers for pricing. Use 5% and 95% percentiles as cutoff

df_pricing_with_counts = df_pricing_with_counts[(df_pricing_with_counts.price>=40)
                                                & (df_pricing_with_counts.price<=550)]

In [28]:
df_pricing_with_counts.shape

(2599154, 8)

In [29]:
# Drill down to houses that price available in 12 months and 7 days a week

df_pricing_available = df_pricing_with_counts[
                                              (df_pricing_with_counts['month_count']==12)
                                              & (df_pricing_with_counts['day_of_week_count']==7)
                                             ]

In [30]:
df_pricing_available.shape

(1497091, 8)

In [62]:
df_pricing_available.price.describe()

count    1.497091e+06
mean     1.652814e+02
std      1.077519e+02
min      4.000000e+01
25%      8.500000e+01
50%      1.350000e+02
75%      2.130000e+02
max      5.500000e+02
Name: price, dtype: float64

In [32]:
df_pricing_available.isnull().sum()

listing_id           0
date                 0
available            0
price                0
month                0
day_of_week          0
month_count          0
day_of_week_count    0
dtype: int64

In [33]:
# Group by month by weekday, and calculate an average pricing

df_pricing_by_month_weekday = df_pricing_available.groupby(['listing_id', 'month', 'day_of_week']) \
                              .price.mean()  \
                              .reset_index()
df_pricing_by_month_weekday.head()

,listing_id,month,day_of_week,price
0,5858,1,0,235.0
1,5858,1,1,235.0
2,5858,1,2,235.0
3,5858,1,3,235.0
4,5858,1,4,235.0


In [34]:
df_pricing_by_month_weekday.describe(include='all')

,listing_id,month,day_of_week,price
count,3.708570e+05,370857.000000,370857.000000,370857.000000
mean,1.350731e+07,6.455588,3.004762,165.770394
std,8.696651e+06,3.462933,2.003016,107.889496
min,5.858000e+03,1.000000,0.000000,40.000000
25%,5.632672e+06,3.000000,1.000000,85.000000
50%,1.378224e+07,6.000000,3.000000,135.000000
75%,2.126187e+07,10.000000,5.000000,215.000000
max,2.815247e+07,12.000000,6.000000,550.000000


In [35]:
# Check for null values if any

df_pricing_by_month_weekday.isnull().sum()

listing_id     0
month          0
day_of_week    0
price          0
dtype: int64

In [36]:
# Rename the average price column to avoid confusion

df_pricing_by_month_weekday.rename(columns={'price':'price_by_month_week'}, inplace=True)

In [37]:
df_pricing_by_month_weekday.shape

(370857, 4)

In [38]:
def get_month(number):
    """
    Given a number, return the name of month.
    
    """
    month_lookup = {1:'Jan.', 2:'Feb.', 3:'Mar.', 4:'Apr.', 5:'May.', 6:'Jun.',
                    7:'Jul.', 8:'Aug.', 9:'Sep.', 10:'Oct.', 11:'Nov.', 12:'Dec.'}
    
    return month_lookup[number]

In [39]:
def get_weekdays(number):
    """
    Given a number, return the name of month.
    
    """
    weekday_lookup = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 
                      4:'Friday', 5:'Saturday', 6:'Sunday'}
    
    return weekday_lookup[number]

In [40]:
df_pricing_by_month_weekday['month'] = df_pricing_by_month_weekday['month']  \
                                       .apply(lambda x: get_month(x))

In [41]:
df_pricing_by_month_weekday['day_of_week'] = df_pricing_by_month_weekday['day_of_week']  \
                                             .apply(lambda x: get_weekdays(x))

In [52]:
# This is to create visual to show seasonality

df_visual = pd.merge(left=df_pricing_by_month_weekday, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [53]:
df_visual.shape

(245629, 37)

In [54]:
df_visual.columns

Index(['listing_id', 'month', 'day_of_week', 'price_by_month_week', 'id',
       'bedrooms', 'cleaning_fee', 'room_type_Entire home/apt',
       'reviews_per_month', 'city_San Francisco', 'city_Oakland', 'bathrooms',
       'accommodates', 'number_of_reviews', 'host_listings_count',
       'review_scores_rating_log10', 'host_yrs', 'guests_included',
       'calculated_host_listings_count', 'review_scores_location_log10',
       'beds', 'yrs_since_first_review', 'host_response_rate',
       'amenities_Air conditioning', 'review_scores_value_log10',
       'jurisdiction_names_Santa Cruz County, CA', 'amenities_Cable TV',
       'neighbourhood_cleansed_Palo Alto', 'host_is_superhost_f',
       'amenities_Lock on bedroom door', 'amenities_Private entrance',
       'amenities_Familykid friendly', 'district_Santa Clara',
       'neighbourhood_cleansed_San Jose', 'host_identity_verified_f',
       'ref_district', 'ref_jurisdiction'],
      dtype='object')

In [65]:
df_visual_entire_apartment = df_visual[df_visual['room_type_Entire home/apt']==1 ]
df_visual_single_room = df_visual[df_visual['room_type_Entire home/apt']!=1 ]

print(df_visual_entire_apartment.shape, df_visual_single_room.shape )

(135866, 37) (109763, 37)


In [79]:
df_visual_entire_apartment.ref_district.describe

<bound method NDFrame.describe of 0         San Francisco
1         San Francisco
2         San Francisco
3         San Francisco
4         San Francisco
5         San Francisco
6         San Francisco
7         San Francisco
8         San Francisco
9         San Francisco
10        San Francisco
11        San Francisco
12        San Francisco
13        San Francisco
14        San Francisco
15        San Francisco
16        San Francisco
17        San Francisco
18        San Francisco
19        San Francisco
20        San Francisco
21        San Francisco
22        San Francisco
23        San Francisco
24        San Francisco
25        San Francisco
26        San Francisco
27        San Francisco
28        San Francisco
29        San Francisco
              ...      
245599      Santa Clara
245600      Santa Clara
245601      Santa Clara
245602      Santa Clara
245603      Santa Clara
245604      Santa Clara
245605      Santa Clara
245606      Santa Clara
245607      Santa Clara
245608

In [80]:
df_visual_entire_apartment_sf = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='San Francisco']
df_visual_entire_apartment_scz = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='Santa Clara']

print(df_visual_entire_apartment_sf.shape, df_visual_entire_apartment_scz.shape )

(56047, 37) (35034, 37)


In [81]:
df_visual_entire_apartment_sf.id.nunique()

695

In [82]:
df_visual_entire_apartment_scz.id.nunique()

422

In [84]:
# Weekly pattern for entire apartment

df_week_entire_apartment = df_visual_entire_apartment.groupby(['ref_district', 'day_of_week'])  \
                                                     .price_by_month_week.mean()
df_week_entire_apartment

ref_district   day_of_week
Oakland        Friday         159.474027
               Monday         154.944938
               Saturday       159.725316
               Sunday         155.161974
               Thursday       155.060959
               Tuesday        154.571370
               Wednesday      154.744414
San Francisco  Friday         211.870899
               Monday         204.897288
               Saturday       211.966729
               Sunday         205.295141
               Thursday       204.853625
               Tuesday        205.165253
               Wednesday      205.049197
Santa Clara    Friday         209.905996
               Monday         208.723240
               Saturday       209.942656
               Sunday         208.205175
               Thursday       209.189709
               Tuesday        209.198270
               Wednesday      209.493972
Santa Cruz     Friday         279.173827
               Monday         265.707357
               Saturday       

In [69]:
# Weekly pattern for single room

df_week_single_room = df_visual_single_room.groupby(['ref_district', 'day_of_week'])  \
                                           .price_by_month_week.mean()
df_week_single_room

ref_district   day_of_week
Oakland        Friday          77.408285
               Monday          75.695416
               Saturday        77.407270
               Sunday          75.643985
               Thursday        75.771951
               Tuesday         75.718651
               Wednesday       75.716536
San Francisco  Friday         121.290236
               Monday         115.501022
               Saturday       121.449297
               Sunday         115.908191
               Thursday       115.840256
               Tuesday        115.686374
               Wednesday      115.818529
Santa Clara    Friday          82.181104
               Monday          80.824664
               Saturday        82.253229
               Sunday          80.811711
               Thursday        80.821351
               Tuesday         80.761868
               Wednesday       80.724975
Santa Cruz     Friday         127.845906
               Monday         122.235648
               Saturday       

In [70]:
# Montly Pattern for entire apartment

df_visual_entire_apartment.groupby(['ref_district', 'month']).price_by_month_week.mean()

ref_district   month
Oakland        Apr.     158.153970
               Aug.     157.264184
               Dec.     153.305877
               Feb.     156.082218
               Jan.     153.080845
               Jul.     159.672266
               Jun.     159.677606
               Mar.     157.705606
               May.     159.250697
               Nov.     151.698238
               Oct.     151.726567
               Sep.     156.919118
San Francisco  Apr.     207.472221
               Aug.     209.898199
               Dec.     203.178067
               Feb.     207.013189
               Jan.     205.855001
               Jul.     209.035882
               Jun.     208.817503
               Mar.     207.347120
               May.     207.832853
               Nov.     202.567577
               Oct.     206.984157
               Sep.     208.986456
Santa Clara    Apr.     211.766231
               Aug.     208.364194
               Dec.     205.188419
               Feb.     210.138842

In [71]:
# Montly Pattern for single room

df_visual_single_room.groupby(['ref_district', 'month']).price_by_month_week.mean()

ref_district   month
Oakland        Apr.      77.677205
               Aug.      76.846752
               Dec.      74.175045
               Feb.      76.617210
               Jan.      74.548583
               Jul.      77.792769
               Jun.      77.792769
               Mar.      77.669665
               May.      77.878307
               Nov.      73.508439
               Oct.      74.034876
               Sep.      75.448894
San Francisco  Apr.     118.221302
               Aug.     120.163396
               Dec.     112.868533
               Feb.     116.927029
               Jan.     114.475622
               Jul.     120.691553
               Jun.     120.156573
               Mar.     118.046094
               May.     119.168788
               Nov.     113.456209
               Oct.     116.937953
               Sep.     117.467900
Santa Clara    Apr.      82.440170
               Aug.      80.775148
               Dec.      79.769163
               Feb.      82.211255

In [42]:
df_pricing_final = pd.get_dummies(df_pricing_by_month_weekday, 
                                  prefix='',
                                  prefix_sep='',
                                  columns=['month', 'day_of_week'])

In [43]:
df_pricing_final.columns

Index(['listing_id', 'price_by_month_week', 'Apr.', 'Aug.', 'Dec.', 'Feb.',
       'Jan.', 'Jul.', 'Jun.', 'Mar.', 'May.', 'Nov.', 'Oct.', 'Sep.',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'],
      dtype='object')

In [44]:
df_pricing_final.shape

(370857, 21)

In [45]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'

pkl_path = pkl_file.format('listings_with_selected_features')

df_listings_reviews = pd.read_pickle(pkl_path)
df_listings_reviews.head()

,id,bedrooms,cleaning_fee,room_type_Entire home/apt,reviews_per_month,city_San Francisco,city_Oakland,bathrooms,accommodates,number_of_reviews,...,neighbourhood_cleansed_Palo Alto,host_is_superhost_f,amenities_Lock on bedroom door,amenities_Private entrance,amenities_Familykid friendly,district_Santa Clara,neighbourhood_cleansed_San Jose,host_identity_verified_f,ref_district,ref_jurisdiction
0,958,1.0,2.00000,1,1.38,1,0,1.0,3,152,...,0,0,0,1,1,0,0,0,San Francisco,SAN FRANCISCO
1,5858,2.0,2.00000,1,0.99,1,0,1.0,5,112,...,0,1,0,1,1,0,0,0,San Francisco,SAN FRANCISCO
2,7918,1.0,1.69897,0,0.16,1,0,4.0,2,17,...,0,1,1,1,0,0,0,0,San Francisco,SAN FRANCISCO
3,8142,1.0,1.69897,0,0.15,1,0,4.0,2,7,...,0,1,1,1,1,0,0,0,San Francisco,SAN FRANCISCO
4,8567,2.0,2.09691,1,0.27,1,0,1.0,6,30,...,0,1,0,0,1,0,0,1,San Francisco,SAN FRANCISCO


In [46]:
df_listings_reviews.shape

(10352, 33)

In [47]:
df_final = pd.merge(left=df_pricing_final, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [48]:
df_final.shape

(245629, 54)

In [49]:
df_final.isnull().sum()

listing_id                                  0
price_by_month_week                         0
Apr.                                        0
Aug.                                        0
Dec.                                        0
Feb.                                        0
Jan.                                        0
Jul.                                        0
Jun.                                        0
Mar.                                        0
May.                                        0
Nov.                                        0
Oct.                                        0
Sep.                                        0
Friday                                      0
Monday                                      0
Saturday                                    0
Sunday                                      0
Thursday                                    0
Tuesday                                     0
Wednesday                                   0
id                                

In [50]:
df_final.shape

(245629, 54)

In [52]:
df_final.columns

Index(['listing_id', 'price_by_month_week', 'Apr.', 'Aug.', 'Dec.', 'Feb.',
       'Jan.', 'Jul.', 'Jun.', 'Mar.', 'May.', 'Nov.', 'Oct.', 'Sep.',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday', 'id', 'bedrooms', 'cleaning_fee',
       'room_type_Entire home/apt', 'reviews_per_month', 'city_San Francisco',
       'city_Oakland', 'bathrooms', 'accommodates', 'number_of_reviews',
       'host_listings_count', 'review_scores_rating_log10', 'host_yrs',
       'guests_included', 'calculated_host_listings_count',
       'review_scores_location_log10', 'beds', 'yrs_since_first_review',
       'host_response_rate', 'amenities_Air conditioning',
       'review_scores_value_log10', 'jurisdiction_names_Santa Cruz County, CA',
       'amenities_Cable TV', 'neighbourhood_cleansed_Palo Alto',
       'host_is_superhost_f', 'amenities_Lock on bedroom door',
       'amenities_Private entrance', 'amenities_Familykid friendly',
       'district_Santa Clara', 'n

In [53]:
# Drop duplicate columns

df_final.drop(['id'], axis=1, inplace=True)

In [54]:
df_final.shape

(245629, 53)

In [55]:
# Create correlation matrix
corr_matrix = df_final.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

to_drop

[]

In [56]:
# Drop features that have high correlation

df_final.drop(to_drop, axis=1, inplace=True)

In [57]:
df_final.shape

(245629, 53)

In [58]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'
pkl_path = pkl_file.format('listings_seasonality')

try: 
    with open(pkl_path, 'rb') as file:
        df_final = pd.read_pickle(file)
except:
    with open(pkl_path, 'wb') as file:
        pd.to_pickle(df_final, file)

In [59]:
# Perform multiple modeling for price prediction

X = df_final.drop(['listing_id', 'price_by_month_week', 'ref_district', 'ref_jurisdiction'], axis=1)
y = df_final['price_by_month_week']

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [61]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [67]:
# Model-LR

model=sm.OLS(y_train, sm.add_constant(X_train_trans))
fit=model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     price_by_month_week   R-squared:                       0.615
Model:                             OLS   Adj. R-squared:                  0.615
Method:                  Least Squares   F-statistic:                     5724.
Date:                 Mon, 17 Sep 2018   Prob (F-statistic):               0.00
Time:                         15:15:42   Log-Likelihood:            -9.6040e+05
No. Observations:               171940   AIC:                         1.921e+06
Df Residuals:                   171891   BIC:                         1.921e+06
Df Model:                           48                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        160.9641      0.156   1032.325      0.000     160.658     161.270
x1          3.084e+10   5.97e+11      0.052      0.959   -1.14e+12     1.2e+12
x2          3.013e+10   5.83e+11      0.052      0.959   -1.11e+12    1.17e+12
x3          3.085e+10   5.97e+11      0.052      0.959   -1.14e+12     1.2e+12
x4          3.102e+10      6e+11      0.052      0.959   -1.15e+12    1.21e+12
x5          3.091e+10   5.98e+11      0.052      0.959   -1.14e+12     1.2e+12
x6          3.097e+10   5.99e+11      0.052      0.959   -1.14e+12    1.21e+12
x7          3.102e+10      6e+11      0.052      0.959   -1.15e+12    1.21e+12
x8           3.09e+10   5.98e+11      0.052      0.959   -1.14e+12     1.2e+12
x9          3.098e+10      6e+11      0.052      0.959   -1.14e+12    1.21e+12
x10         3.064e+10   5.93e+11      0.052      0.959   -1.13e+12    1.19e+12
x11         3.034e+10   5.87e+11      0.052      0.959   -1.12e+12    1.18e+12
x12         2.898e+10   5.61e+11      0.052      0.959   -1.07e+12    1.13e+12
x13         2.947e+13   1.43e+13      2.066      0.039    1.51e+12    5.74e+13
x14         2.964e+13   1.43e+13      2.066      0.039    1.52e+12    5.78e+13
x15         2.953e+13   1.43e+13      2.066      0.039    1.51e+12    5.76e+13
x16         2.962e+13   1.43e+13      2.066      0.039    1.52e+12    5.77e+13
x17         2.955e+13   1.43e+13      2.066      0.039    1.51e+12    5.76e+13
x18         2.948e+13   1.43e+13      2.066      0.039    1.51e+12    5.75e+13
x19         2.939e+13   1.42e+13      2.066      0.039    1.51e+12    5.73e+13
x20           29.7703      0.258    115.487      0.000      29.265      30.276
x21            5.8784      0.192     30.681      0.000       5.503       6.254
x22           26.0485      0.217    120.240      0.000      25.624      26.473
x23           -9.3136      0.274    -33.952      0.000      -9.851      -8.776
x24           20.5460      0.882     23.296      0.000      18.817      22.275
x25           -2.1887      0.606     -3.612      0.000      -3.376      -1.001
x26           -0.4454      0.182     -2.450      0.014      -0.802      -0.089
x27           32.2288      0.350     92.098      0.000      31.543      32.915
x28           -1.4237      0.299     -4.766      0.000      -2.009      -0.838
x29            4.3824      0.222     19.734      0.000       3.947       4.818
x30           -0.1583      0.338     -0.468      0.640      -0.822       0.505
x31           -2.2453      0.207    -10.838      0.000      -2.651      -1.839
x32            4.0290      0.196     20.603      0.000       3.646       4.412
x33           -8.7025      0.228    -38.221      0.000      -9.149      -8.256
x34           11.0738      0.351     31.511      0.000      10.385      11.763
x35           -8.8916      0.314    -28.337      0.000      -9.507    

In [68]:
# Model-Random Forest

rf = RandomForestRegressor(n_estimators=200, 
                           criterion='mse', 
                           random_state=3, 
                           n_jobs=-1)
rf.fit(X_train_trans, y_train)

y_train_pred = rf.predict(X_train_trans)
y_test_pred = rf.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 6.404, test: 46.639
R^2 train: 0.999, test: 0.996


In [69]:
df_final.listing_id.nunique()

3006

In [70]:
# Double check on price range. The range is as expected

y.describe(include='all')

count    245629.000000
mean        161.184664
std         104.196466
min          40.000000
25%          85.000000
50%         129.000000
75%         210.000000
max         550.000000
Name: price_by_month_week, dtype: float64

In [71]:
# Map feature importance with labels

feature_labels = X.columns
feature_tuples = zip (feature_labels, rf.feature_importances_)
feature_lists = [list(feature_tuple) for feature_tuple in feature_tuples]

In [72]:
# Order features based on importance

feature_lists.sort(key=lambda x: abs(x[1]), reverse=True)
feature_lists

[['bedrooms', 0.4156324736485746],
 ['cleaning_fee', 0.07034290324560516],
 ['room_type_Entire home/apt', 0.06705745536425838],
 ['accommodates', 0.061629916824698594],
 ['reviews_per_month', 0.04828570528371509],
 ['host_listings_count', 0.04310400183024202],
 ['number_of_reviews', 0.030599769313474728],
 ['bathrooms', 0.026573235736360914],
 ['review_scores_rating_log10', 0.02514303952043061],
 ['calculated_host_listings_count', 0.02201171941033038],
 ['host_yrs', 0.01789310282225505],
 ['city_San Francisco', 0.017425373050930167],
 ['host_response_rate', 0.01506220643000437],
 ['yrs_since_first_review', 0.011552273510368243],
 ['guests_included', 0.010881599402600497],
 ['jurisdiction_names_Santa Cruz County, CA', 0.010464262456094046],
 ['beds', 0.009429364298248035],
 ['city_Oakland', 0.009412498980338828],
 ['host_is_superhost_f', 0.008795707369316634],
 ['host_identity_verified_f', 0.007822633962941604],
 ['review_scores_location_log10', 0.0075009318936435695],
 ['review_scores_

In [73]:
features = [list_[0] for list_ in feature_lists]
features

['bedrooms',
 'cleaning_fee',
 'room_type_Entire home/apt',
 'accommodates',
 'reviews_per_month',
 'host_listings_count',
 'number_of_reviews',
 'bathrooms',
 'review_scores_rating_log10',
 'calculated_host_listings_count',
 'host_yrs',
 'city_San Francisco',
 'host_response_rate',
 'yrs_since_first_review',
 'guests_included',
 'jurisdiction_names_Santa Cruz County, CA',
 'beds',
 'city_Oakland',
 'host_is_superhost_f',
 'host_identity_verified_f',
 'review_scores_location_log10',
 'review_scores_value_log10',
 'amenities_Cable TV',
 'amenities_Lock on bedroom door',
 'amenities_Familykid friendly',
 'amenities_Air conditioning',
 'neighbourhood_cleansed_San Jose',
 'amenities_Private entrance',
 'Saturday',
 'Friday',
 'Sep.',
 'Nov.',
 'neighbourhood_cleansed_Palo Alto',
 'Dec.',
 'Oct.',
 'Aug.',
 'Jul.',
 'Jun.',
 'district_Santa Clara',
 'Jan.',
 'May.',
 'Feb.',
 'Mar.',
 'Apr.',
 'Sunday',
 'Tuesday',
 'Thursday',
 'Monday',
 'Wednesday']

In [74]:
columns =['listing_id']
columns.extend(features)
columns.extend(['ref_district', 'ref_jurisdiction', 'price_by_month_week'])

In [76]:
df_data = df_final[columns]
df_data.head()

,listing_id,bedrooms,cleaning_fee,room_type_Entire home/apt,accommodates,reviews_per_month,host_listings_count,number_of_reviews,bathrooms,review_scores_rating_log10,...,Mar.,Apr.,Sunday,Tuesday,Thursday,Monday,Wednesday,ref_district,ref_jurisdiction,price_by_month_week
0,5858,2.0,2.0,1,5,0.99,2,112,1.0,1.991226,...,0,0,0,0,0,1,0,San Francisco,SAN FRANCISCO,235.0
1,5858,2.0,2.0,1,5,0.99,2,112,1.0,1.991226,...,0,0,0,1,0,0,0,San Francisco,SAN FRANCISCO,235.0
2,5858,2.0,2.0,1,5,0.99,2,112,1.0,1.991226,...,0,0,0,0,0,0,1,San Francisco,SAN FRANCISCO,235.0
3,5858,2.0,2.0,1,5,0.99,2,112,1.0,1.991226,...,0,0,0,0,1,0,0,San Francisco,SAN FRANCISCO,235.0
4,5858,2.0,2.0,1,5,0.99,2,112,1.0,1.991226,...,0,0,0,0,0,0,0,San Francisco,SAN FRANCISCO,235.0


In [77]:
df_data.shape

(245629, 53)

In [78]:
df_data.rename(columns={'price_by_month_week':'price'}, inplace=True)

In [79]:
df_data.isnull().sum()

listing_id                                  0
bedrooms                                    0
cleaning_fee                                0
room_type_Entire home/apt                   0
accommodates                                0
reviews_per_month                           0
host_listings_count                         0
number_of_reviews                           0
bathrooms                                   0
review_scores_rating_log10                  0
calculated_host_listings_count              0
host_yrs                                    0
city_San Francisco                          0
host_response_rate                          0
yrs_since_first_review                      0
guests_included                             0
jurisdiction_names_Santa Cruz County, CA    0
beds                                        0
city_Oakland                                0
host_is_superhost_f                         0
host_identity_verified_f                    0
review_scores_location_log10      

In [93]:
temp_df = df_data[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].copy()

days = temp_df.apply(lambda x: x.idxmax(), axis=1)
days

0            Monday
1           Tuesday
2         Wednesday
3          Thursday
4            Friday
5          Saturday
6            Sunday
7            Monday
8           Tuesday
9         Wednesday
10         Thursday
11           Friday
12         Saturday
13           Sunday
14           Monday
15          Tuesday
16        Wednesday
17         Thursday
18           Friday
19         Saturday
20           Sunday
21           Monday
22          Tuesday
23        Wednesday
24         Thursday
25           Friday
26         Saturday
27           Sunday
28           Monday
29          Tuesday
            ...    
245599     Saturday
245600       Sunday
245601       Monday
245602      Tuesday
245603    Wednesday
245604     Thursday
245605       Friday
245606     Saturday
245607       Sunday
245608       Monday
245609      Tuesday
245610    Wednesday
245611     Thursday
245612       Friday
245613     Saturday
245614       Sunday
245615       Monday
245616      Tuesday
245617    Wednesday


In [ ]:
csv_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

csv_path = csv_file.format('airbnb_data')

df_data.to_csv(csv_path)